# Appendix A code

In [18]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

## CPU

### Model

Listing A.4 A multilayer perceptron with two hidden layers

In [4]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            # 1st hidden layer
            torch.nn.Linear(num_inputs,30),
            torch.nn.ReLU(),
            # 2nd hidden layer
            torch.nn.Linear(30,20),
            torch.nn.ReLU(),
            # output layer
            torch.nn.Linear(20,num_outputs)
        )

    def forward(self,x):
        logits = self.layers(x)
        return logits

init a model

In [5]:
model = NeuralNetwork(50,3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


### Data

Listing A.5 Creating a toy dataset

In [8]:
x_train = torch.tensor([
    [-1.2,3.1],
    [-0.9,2.9],
    [-0.5,2.6],
    [2.3,-1.1],
    [2.7,-1.5]
])
y_train = torch.tensor([0,0,0,1,1])

x_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])
y_test = torch.tensor([0,1])

Listing A.6 Defining a custom Dataset class

In [14]:
class ToyDataset(Dataset):
    def __init__(self, X, Y):
        self.features = X
        self.labels = Y
        
    def __getitem__(self,index):
        x = self.features[index]
        y = self.labels[index]
        return x, y
        
    def __len__(self):
        return self.labels.shape[0]

In [15]:
train_ds = ToyDataset(x_train, y_train)
test_ds = ToyDataset(x_test, y_test)

Listing A.7 Init data loaders

In [40]:
train_loader = DataLoader(
    dataset = train_ds,
    batch_size = 2,
    shuffle = True,
    num_workers = 0,
    drop_last = True
)

test_loader = DataLoader(
    dataset = test_ds,
    batch_size = 2,
    shuffle = False,
    num_workers = 0
)

### Training

Listing A.9 A toy training cycle

In [77]:
torch.manual_seed(1)
# construct the model and the optimizer
model = NeuralNetwork(num_inputs = 2, num_outputs = 2)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.5)

# train
num_epoch = 4
for epoch in range(num_epoch):
    # set model in training mode
    model.train()

    # batches
    for batch_index, (features, labels) in enumerate(train_loader):
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        # optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        #logging
        print(f"Epoch: {epoch+1:03d}/{num_epoch:03d}"
        f" | Batch {batch_index:03d}/{len(train_loader):03d}"
        f" | Train Loss: {loss:.2f}")

    model.eval()

Epoch: 001/004 | Batch 000/002 | Train Loss: 1.12
Epoch: 001/004 | Batch 001/002 | Train Loss: 0.72
Epoch: 002/004 | Batch 000/002 | Train Loss: 0.18
Epoch: 002/004 | Batch 001/002 | Train Loss: 0.08
Epoch: 003/004 | Batch 000/002 | Train Loss: 0.01
Epoch: 003/004 | Batch 001/002 | Train Loss: 0.01
Epoch: 004/004 | Batch 000/002 | Train Loss: 0.01
Epoch: 004/004 | Batch 001/002 | Train Loss: 0.00


model test

In [66]:
torch.argmax(model(x_train),dim=1)
torch.argmax(model(x_train),dim=1)==y_train
compare = torch.argmax(model(x_train),dim=1)==y_train
print(compare)
print(5+torch.sum(compare))
print(len(compare))

tensor([True, True, True, True, True])
tensor(10)
5


### Validating

Listing A.10 Compute prediction accuracy

In [72]:
def compute_accuracy(model,dataloader):
    model = model.eval()
    correct = 0.0
    total_example = 0.0

    for batch_index, (features, labels) in enumerate(dataloader):
        # forward
        with torch.no_grad():
            logits = model(features)

        # validate
        predictions = torch.argmax(logits, dim=1)
        '''print(f"predictions: {predictions}"
            f" | labels: {labels}")'''
        compare = predictions == labels
        correct +=torch.sum(compare)
        total_example += len(compare)

    return (correct/total_example).item()

In [74]:
print(compute_accuracy(model, train_loader))
print(compute_accuracy(model, test_loader))

1.0
1.0


## GPU

### GPU info

In [78]:
# GPU available
print(f"GPU is available: {torch.cuda.is_available()}")
# GPU ranks
print(f"#GPU = {torch.cuda.device_count()}")

GPU is available: True
#GPU = 1


In [79]:
if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    print(f"GPUs available: {gpu_count}\n")

    for i in range(gpu_count):
        name = torch.cuda.get_device_name(i)
        props = torch.cuda.get_device_properties(i)

        print(f"--- GPU {i} ---")
        print(f"Name: {name}")
        print(f"Total VRAM: {props.total_memory / (1024**3):.2f} GB")
        print(f"Compute Capability: {props.major}.{props.minor}")
        print(f"Multi-Processor Count: {props.multi_processor_count}")
else:
    print("No CUDA-compatible GPU detected.")


GPUs available: 1

--- GPU 0 ---
Name: NVIDIA RTX A400
Total VRAM: 4.00 GB
Compute Capability: 8.6
Multi-Processor Count: 6


## Generalized training

In [81]:
torch.manual_seed(1)

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# construct the model and the optimizer
model = NeuralNetwork(num_inputs = 2, num_outputs = 2)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.5)

# send modelto device
model.to(device)

# train
num_epoch = 4
for epoch in range(num_epoch):
    # set model in training mode
    model.train()

    # batches
    for batch_index, (features, labels) in enumerate(train_loader):
        # send dataset to device
        features, labels = features.to(device), labels.to(device)
        
        logits = model(features)
        loss = F.cross_entropy(logits, labels)
        # optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        #logging
        print(f"Epoch: {epoch+1:03d}/{num_epoch:03d}"
        f" | Batch {batch_index:03d}/{len(train_loader):03d}"
        f" | Train Loss: {loss:.2f}")

    model.eval()

Epoch: 001/004 | Batch 000/002 | Train Loss: 1.12
Epoch: 001/004 | Batch 001/002 | Train Loss: 0.72
Epoch: 002/004 | Batch 000/002 | Train Loss: 0.18
Epoch: 002/004 | Batch 001/002 | Train Loss: 0.08
Epoch: 003/004 | Batch 000/002 | Train Loss: 0.01
Epoch: 003/004 | Batch 001/002 | Train Loss: 0.01
Epoch: 004/004 | Batch 000/002 | Train Loss: 0.01
Epoch: 004/004 | Batch 001/002 | Train Loss: 0.00
